In [1]:
import json
import os
import warnings
warnings.filterwarnings(action='ignore')

from tqdm import tqdm
from glob import glob

import numpy as np
import pandas as pd

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
th = 0.01

detections = [
    'ablation/trt8x-orin64/ts-fp32/results-test_dev.json',
    'ablation/trt8x-orin64/trt-fp32/results-test_dev.json',
    'ablation/trt8x-orin64/trt-fp16/results-test_dev.json',
    'ablation/trt8x-orin64/trt-int8/results-test_dev.json',
    
]

gt_path = 'data/SeaDronesSee/test_dev.json'

all_metrics = []

names = ['weights', 'FPS', 'AP', 'AP50', 'AP75', 'APu', 'APvt', 'APt', 'APs', 'APm', 'APl', 
         'AR1', 'AR10', 'AR100', 'ARu', 'ARvt', 'ARt', 'ARs', 'ARm', 'ARl']

for det_path in detections:
    if not os.path.isfile(det_path):
        print(det_path)
        continue


    det = json.load(open(det_path))
    print(len(det))
    det = [x for x in det if x['score'] >= th] if th is not None else det
    print(len(det))

    th_path = det_path.replace('.json', f'-mapped.json')
    with open(th_path, 'w', encoding='utf-8') as f:
        json.dump(det, f, ensure_ascii=False, indent=4)

    weights = os.path.dirname(det_path).split(os.sep)[-1]
    print(weights)
    t = [x.rstrip() for x in open(det_path.replace(os.path.basename(det_path), 'times.txt'))][0]
    fps = 1/float(t)
    print(t, fps)
    anno = COCO(gt_path)

    pred = anno.loadRes(th_path)  # init predictions api
    eval = COCOeval(anno, pred, 'bbox') #, [100, 500])
    # eval = COCOeval(anno, pred, 'bbox')

    imgIds = sorted(anno.getImgIds())
    eval.params.imgIds = imgIds

    eval.evaluate()
    eval.accumulate()
    eval.summarize()
    map, map50 = eval.stats[:2]

    stats = {k:v for k,v in zip(names, [weights, fps] + [x*100 for x in eval.stats.tolist()])}
    all_metrics.append(stats)

metrics = pd.DataFrame(all_metrics)
metrics.to_csv(f'ablation/SDS-trt85-orin64.csv', index=False)

81103
81103
ts-fp32
0.08207161117688808 12.18448115810363
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
Loading and preparing results...
DONE (t=36.65s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=15.97s).
Accumulating evaluation results...
DONE (t=3.05s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.531
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.850
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.577
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= micro | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=v-tiny | maxDets=100 ] = 0.343
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=  tiny | maxDets=100 ] = 0.521
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.617
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.374
 Ave

In [3]:
metrics

,weights,FPS,AP,AP50,AP75,APu,APvt,APt,APs,APm,APl,AR1,AR10,AR100,ARu,ARvt,ARt,ARs,ARm,ARl
0,ts-fp32,12.184481,53.069107,84.985893,57.680463,-100.0,34.300898,52.144734,61.689703,37.380589,70.820645,26.934328,62.040071,62.071732,-100.0,40.842382,62.635249,69.910543,43.657638,75.647066
1,trt-fp32,28.189748,53.083745,84.968709,57.694275,-100.0,34.287990,52.121078,61.688177,37.422045,71.325373,26.939628,62.050799,62.080483,-100.0,40.869377,62.590135,69.920486,43.775709,75.876073
2,trt-fp16,35.090082,53.101449,85.031319,57.686025,-100.0,34.366116,52.140449,61.707081,37.518998,71.095956,26.946671,62.057455,62.087475,-100.0,40.925362,62.617557,69.921420,43.719676,76.085997
3,trt-int8,39.379594,28.637995,56.077935,27.107017,-100.0,25.883434,31.088758,25.651812,18.778118,49.456406,19.946456,45.033318,45.055439,-100.0,34.733005,45.998122,46.254533,30.361688,50.876073


In [4]:
from tabulate import tabulate

out_table = 'ablation/trt8x-orin64/table.tex'

header = metrics.columns.tolist()

rows = []
for i in range(len(metrics)):
    row = metrics.iloc[i].values.tolist()
    formatted_row = []
    for j,x in enumerate(row):
        if isinstance(x, str) or isinstance(x, np.int64):
            formatted_row.append(x)
            continue
        elif isinstance(x, float):
            formatted_row.append(f'{x:.1f}')
        else:
            print(type(x))

    rows.append(formatted_row)

open(out_table, 'w').write(tabulate(rows, header, tablefmt='latex_raw', stralign='center', numalign='left', floatfmt=".1f"))
print(tabulate(rows, header))

weights      FPS    AP    AP50    AP75    APu    APvt    APt    APs    APm    APl    AR1    AR10    AR100    ARu    ARvt    ARt    ARs    ARm    ARl
---------  -----  ----  ------  ------  -----  ------  -----  -----  -----  -----  -----  ------  -------  -----  ------  -----  -----  -----  -----
ts-fp32     12.2  53.1    85      57.7   -100    34.3   52.1   61.7   37.4   70.8   26.9    62       62.1   -100    40.8   62.6   69.9   43.7   75.6
trt-fp32    28.2  53.1    85      57.7   -100    34.3   52.1   61.7   37.4   71.3   26.9    62.1     62.1   -100    40.9   62.6   69.9   43.8   75.9
trt-fp16    35.1  53.1    85      57.7   -100    34.4   52.1   61.7   37.5   71.1   26.9    62.1     62.1   -100    40.9   62.6   69.9   43.7   76.1
trt-int8    39.4  28.6    56.1    27.1   -100    25.9   31.1   25.7   18.8   49.5   19.9    45       45.1   -100    34.7   46     46.3   30.4   50.9
